In [ ]:
# TO USE, upload an api key from Kaggle (kaggle.json)
# Runtime -> Change runtime type -> Hardware accelerator set to GPU (optional)

In [ ]:
!pip install tensorflow
!pip install keras
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

In [3]:
!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d gpiosenka/100-bird-species

import zipfile
local_zip = '/content/100-bird-species.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

100% 1.69G/1.69G [00:43<00:00, 43.2MB/s]
100% 1.69G/1.69G [00:43<00:00, 42.0MB/s]


In [ ]:
# Generate batches of tensor image data with real-time data augmentation.
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

train_preprocessor = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        shear_range=0.2,
        rotation_range=5,
        zoom_range=0.15,
        horizontal_flip=True)
train_generator = train_preprocessor.flow_from_directory('/content/train',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical'
                                                    shuffle=True)

val_preprocessor = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_preprocessor.flow_from_directory('/content/valid',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')

test_preprocessor = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_preprocessor.flow_from_directory('/content/test',
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 70626 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.


In [ ]:
# adapted from https://learning.oreilly.com/library/view/hands-on-machine-learning/9781098125967/ch14.html#idm45983091867328
# and https://learning.oreilly.com/library/view/hands-on-machine-learning/9781098125967/ch11.html#idm45983104434448

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D

# download base model
base_model = ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
# freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False


n_classes = 450
model=Sequential()
model.add(base_model)
model.add(Flatten(input_shape=[224, 224]))
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(rate=0.2))
model.add(Dense(2048, activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(rate=0.2))
model.add(Dense(n_classes, activation="softmax"))


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

model.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 2048)              205522944 
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                         

In [ ]:
history = model.fit(train_generator, 
                    validation_data=val_generator, 
                    epochs=3, 
                    workers=10,
                    use_multiprocessing=True)

Epoch 1/3
1104/1104 [==============================] - 848s 750ms/step - loss: 2.5213 - accuracy: 0.4748 - val_loss: 0.5845 - val_accuracy: 0.8471
Epoch 2/3
1104/1104 [==============================] - 808s 724ms/step - loss: 0.9307 - accuracy: 0.7638 - val_loss: 0.3768 - val_accuracy: 0.8924
Epoch 3/3
1104/1104 [==============================] - 808s 725ms/step - loss: 0.6692 - accuracy: 0.8263 - val_loss: 0.3418 - val_accuracy: 0.9062


In [ ]:
# top layers are now relatively well-trained (above 80% accuracy), so unfreeze the
# base_model's layers, reduce learning rate, recompile, and continue training
for layer in base_model.layers[56:]:
    layer.trainable = True

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_generator, 
                    validation_data=val_generator, 
                    epochs=10, 
                    workers=10,
                    use_multiprocessing=True)

Epoch 1/10
1104/1104 [==============================] - 848s 754ms/step - loss: 0.4228 - accuracy: 0.8866 - val_loss: 0.2118 - val_accuracy: 0.9400
Epoch 2/10
1104/1104 [==============================] - 835s 749ms/step - loss: 0.2292 - accuracy: 0.9351 - val_loss: 0.2124 - val_accuracy: 0.9431
Epoch 3/10
1104/1104 [==============================] - 840s 752ms/step - loss: 0.1759 - accuracy: 0.9485 - val_loss: 0.2160 - val_accuracy: 0.9462
Epoch 4/10
1104/1104 [==============================] - 835s 750ms/step - loss: 0.1360 - accuracy: 0.9590 - val_loss: 0.1752 - val_accuracy: 0.9511
Epoch 5/10
1104/1104 [==============================] - 832s 746ms/step - loss: 0.1119 - accuracy: 0.9663 - val_loss: 0.1666 - val_accuracy: 0.9618
Epoch 6/10
1104/1104 [==============================] - 825s 740ms/step - loss: 0.1021 - accuracy: 0.9698 - val_loss: 0.1850 - val_accuracy: 0.9524
Epoch 7/10
1104/1104 [==============================] - 828s 741ms/step - loss: 0.0853 - accuracy: 0.9742 - val_

In [ ]:
model.evaluate(test_generator,
               use_multiprocessing=True,
               workers=10)

71/71 [==============================] - 11s 139ms/step - loss: 0.1142 - accuracy: 0.9684


[0.11417487263679504, 0.9684444665908813]

In [ ]:
# save trained model
model.save('trained_bird_CNN')

In [ ]:
# zip saved model
!zip -r /content/trained_bird_CNN.zip /content/trained_bird_CNN

from google.colab import files
files.download("/content/trained_bird_CNN.zip")

  adding: content/trained_bird_CNN/ (stored 0%)
  adding: content/trained_bird_CNN/variables/ (stored 0%)
  adding: content/trained_bird_CNN/variables/variables.index (deflated 80%)
  adding: content/trained_bird_CNN/variables/variables.data-00000-of-00001 (deflated 16%)
  adding: content/trained_bird_CNN/assets/ (stored 0%)
  adding: content/trained_bird_CNN/saved_model.pb (deflated 92%)
  adding: content/trained_bird_CNN/keras_metadata.pb (deflated 96%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# https://keras.io/api/preprocessing/image/
# function to print resized input image and model's top three predictions

def classify(img_path, model, label_map, top_n=3):
  img = image.load_img(img_path, grayscale=False, target_size=(224, 224))
  input_arr = image.img_to_array(img)
  input_arr = np.expand_dims(input_arr, axis=0)
  input_arr = preprocess_input(input_arr) # preprocess function specific to ResNet50

  predictions = model.predict(input_arr)

  top_n_indices = np.argpartition(predictions[0], -top_n)[-top_n:] # gets top_n prediction indices
  top_n_indices = top_n_indices[np.argsort(predictions[0][top_n_indices])] # sorts by prediction confidence
  top_n_indices = top_n_indices[::-1] # reverses array so that sorted most to least confident


  for index in top_n_indices:
    print(label_map[index], predictions[0][index])
  plt.matshow(img)

In [ ]:
label_map = (test_generator.class_indices) # maps index in predict array to species name
label_map = {v: k for k, v in label_map.items()} # inverts key-value pairs so that index is key

# for human evaluation of model performance on test images

species_name = "SCARLET MACAW"
import os
dir = "/content/test/" + species_name
for filename in os.listdir(dir):
  if filename.endswith("jpg"): 
    img_path = dir + "/" + filename
    print("\n" + img_path)
    classify(img_path, model, label_map, top_n=3)

In [ ]:
classify("/content/african firefinch.jpeg", model, label_map, top_n=3)